## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.1203,-134.9692,79.52,66,1,6.93,PF,2021-01-16 03:10:03
1,1,Mataura,-46.1927,168.8643,73.00,63,0,8.99,NZ,2021-01-16 03:10:04
2,2,Hermanus,-34.4187,19.2345,57.00,93,2,8.63,ZA,2021-01-16 03:10:04
3,3,Udachnyy,66.4167,112.4000,-46.73,75,69,5.93,RU,2021-01-16 03:10:04
4,4,Qaanaaq,77.4840,-69.3632,9.16,91,98,1.59,GL,2021-01-16 03:10:04


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.1203,-134.9692,79.52,66,1,6.93,PF,2021-01-16 03:10:03
11,11,Byron Bay,-28.6500,153.6167,82.40,65,75,13.80,AU,2021-01-16 03:10:05
12,12,Wanaka,-44.7000,169.1500,78.51,24,2,7.96,NZ,2021-01-16 03:09:57
23,23,Mahebourg,-20.4081,57.7000,78.80,89,20,6.91,MU,2021-01-16 03:10:06
24,24,Trat,12.5000,102.5000,86.00,78,0,5.75,TH,2021-01-16 03:10:06
28,28,Pochutla,15.7432,-96.4661,75.20,73,40,1.25,MX,2021-01-16 03:10:07
32,32,Bonthe,7.5264,-12.5050,76.75,89,8,3.85,SL,2021-01-16 03:10:07
33,33,Lorengau,-2.0226,147.2712,85.93,68,100,8.21,PG,2021-01-16 03:10:07
40,40,Faanui,-16.4833,-151.7500,81.81,74,27,11.68,PF,2021-01-16 03:10:09
41,41,Hithadhoo,-0.6000,73.0833,82.22,75,87,14.29,MV,2021-01-16 03:10:09


In [19]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

164


City_ID       164
City          164
Lat           164
Lng           164
Max Temp      164
Humidity      164
Cloudiness    164
Wind Speed    164
Country       164
Date          164
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,79.52,-23.1203,-134.9692,
11,Byron Bay,AU,82.40,-28.6500,153.6167,
12,Wanaka,NZ,78.51,-44.7000,169.1500,
23,Mahebourg,MU,78.80,-20.4081,57.7000,
24,Trat,TH,86.00,12.5000,102.5000,
28,Pochutla,MX,75.20,15.7432,-96.4661,
32,Bonthe,SL,76.75,7.5264,-12.5050,
33,Lorengau,PG,85.93,-2.0226,147.2712,
40,Faanui,PF,81.81,-16.4833,-151.7500,
41,Hithadhoo,MV,82.22,-0.6000,73.0833,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [23]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(15)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,79.52,-23.1203,-134.9692,Pension Maro'i
11,Byron Bay,AU,82.40,-28.6500,153.6167,Backpackers Inn on the Beach
12,Wanaka,NZ,78.51,-44.7000,169.1500,Wanaka Homestead Lodge and Cottages
23,Mahebourg,MU,78.80,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
24,Trat,TH,86.00,12.5000,102.5000,สวนหนิง-อัศ
28,Pochutla,MX,75.20,15.7432,-96.4661,Hotel Posada San Jose
32,Bonthe,SL,76.75,7.5264,-12.5050,Bonthe Holiday Village
33,Lorengau,PG,85.93,-2.0226,147.2712,Lorengau Harbourside Hotel
40,Faanui,PF,81.81,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
41,Hithadhoo,MV,82.22,-0.6000,73.0833,Scoop Guest House


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))